L'objectif de ce notebook est de fusionner les différents csv des procès verbaux en un seul. Chaque ligne correspondra à un étudiant, une cours et une session (et donc une seule filièere) et aura la structure suivante :

| num_etudiant | niveau | formation | id_cours |   annee   | session | note | pt_jury | code_admission | ects | annee_arrivee_fac | date_naissance | dep_naissance | ville_naissance | francais | toulousain |
|:------------:|:------:|:---------:|:--------:|:---------:|:-------:|:----:|:-------:|:--------------:|:----:|:-----------------:|:--------------:|:-------------:|:---------------:|:--------:|:----------:|
|   12345678   |   L2   |   Maths   | resultat | 2018/2019 |    1    |  12  |    0    |      "ab"      |  60  |        2017       |   01/01/1999   |      031      |      muret      |   True   |    True    |
|   23456789   |   M1   |    SID    |  epmat1s | 2017/2018 |    1    |  13  |    0    |      "ab"      |  30  |        2016       |   01/01/1997   |      069      |       lyon      |   True   |    False   |
|   34567890   |   L3   |     E     | epmat1am | 2016/2017 |    2    |   9  |    0    |      "aj"      |   6  |        2012       |   01/01/1990   |      132      |     londres     |   False  |    False   |
|   12345678   |   L2   |   Maths   |  epmat1s | 2018/2019 |    1    |  11  |    0    |       "p"      |  30  |        2017       |   01/01/1999   |      031      |      muret      |   True   |    True    |

Attention :

* Un étudiant apparaitra plusieurs fois dans ce tableau
* la colonne toulousain indique si un étudiant est né en Haute-Garonne (et donc porbablement à Toulouse ou en périférie) pas s'il est réellement né à Toulouse.

In [5]:
import pandas as pd
from collections import Counter
from tqdm import tqdm

On récupère les fichiers :

In [6]:
from os import listdir
from os.path import isfile, join

data_path = "fichiers_csv/"

PVs = [f for f in listdir(data_path) if isfile(join(data_path, f))]

Initialiser le dataframe avec les colonnes

In [3]:
columns = ["num_etudiant", "niveau", "formation", "id_cours", "annee", "session", "note", "pt_jury", "code_admission", "ects", "annee_arrivee_fac", "date_naissance", "dep_naissance", "ville_naissance", "francais", "toulousain"]
df = pd.DataFrame(columns=columns)

Quelques fonctions

In [4]:
def get_infos(PV_file_name) :
    # Récupère le niveau, la formation le numéro de session et l'année scolaire
    # d'un PV a partir de son nom de fichier
    file_name, extention = PV_file_name.split(".")
    splited = file_name.split(" ")
    niveau = splited[1]
    formation = " ".join(splited[2:-2])
    session, annee = splited[-2:]
    session_num = session[1] # originellement de la forme S2, on veut juste le 2
    annee_principale = annee.split("-")[0]
    annee_scol = f"{annee_principale}/{int(annee_principale)+1}"
    return niveau, formation, session_num, annee_scol

def isInt(number) :
    # Vérifie qu'une valeur (str) est effectivement un int
    try :
        int(number)
        return True
    except ValueError :
        return False

On va ajouter les informations au nouveau dataframe :
(temps d'execution sur Google Colab : ~50 minutes)

In [7]:
for PV in tqdm(PVs, desc="PVs traités ") :
    df_PV = pd.read_csv(join(data_path, PV), delimiter=";")

    # On indique quelles colonnes on considère comme un index
    # (les lignes qui seront dupliquées)
    id_cols = ["num_etudiant", "identification"]

    # On indique quelles colonnes on considère comme des valeurs
    # C'est a dire qu'on va faire fondre notre dataframe par rapport a ces 
    # colonnes. 
    # Lire https://ichi.pro/fr/comprendre-pandas-melt-pd-melt-59551147147557 
    # pour approfondir
    values_cols = [col_name for col_name in df_PV.columns if col_name not in id_cols]

    # on fait "fondre" notre dataframe
    df_melted = df_PV.melt(id_vars = id_cols, value_vars=values_cols, var_name="id_cours", value_name="infos_notes")

    # On récupère les infomations générales du PV
    niveau, formation, session_num, annee_scol = get_infos(PV)

    for _, row in df_melted.iterrows():

        # On récupère l'identification de l'étudiant
        num_etudiant, annee_arrivee_fac, date_naissance, dep_naissance, ville_naissance, francais, toulousain = eval(row["identification"])
        
        # On récupère l'id du cours actuel
        id_cours = row["id_cours"]

        # On récupères les notes et les infos sur les résultats du cours
        note, pt_jury, code_admission, ects, mention, annee_valid, session_valid = eval(row["infos_notes"])
            

        annee_valid = f"{annee_valid}/{int(annee_valid)+1}" if annee_valid is not None and isInt(annee_valid) else None

        # On modifie l'année is dans le PV, la date de validation est spécifiée et
        # est différente de l'année du PV
        annee = annee_scol if (annee_valid == annee_scol or annee_valid is None) else annee_valid
        # On fait de même pour la session
        session = session_num if (session_valid == session_num or session_valid is None) else session_valid
        
        # On créé notre nouvelle ligne pour le nouveau dataframe
        new_row = [num_etudiant, niveau, formation, id_cours, annee, session, note, pt_jury, code_admission, ects, annee_arrivee_fac, date_naissance, dep_naissance, ville_naissance, francais, toulousain]
        df.loc[len(df.index)] = new_row

PVs traités : 100%|██████████| 89/89 [50:28<00:00, 34.03s/it]


In [25]:
len(df)-len(df.drop_duplicates()) # Le nombre de lignes dupliquées
# Cela survient surement car un étudiant redouble mais a validé une matière lors
# de l'année précédente, on peut donc enlever ces lignes

0

In [31]:
import numpy as np

In [34]:
# On elève les lignes dupliquées
df = df.drop_duplicates()
# On remplace les valeurs None par np.nan
df = df.fillna(value=np.nan)
df = df.reset_index(drop=True)

In [35]:
df

,num_etudiant,niveau,formation,id_cours,annee,session,note,pt_jury,code_admission,ects,annee_arrivee_fac,date_naissance,dep_naissance,ville_naissance,francais,toulousain
0,21300160,L3,E,resultat,2016/2017,2,NaN,NaN,def,NaN,2013,06/06/1995,031,toulouse,True,True
1,21005127,L3,E,resultat,2016/2017,2,10.425,NaN,adm,60.0,2010,06/03/1992,031,toulouse,True,True
2,21301391,L3,E,resultat,2016/2017,2,8.357,NaN,aj,NaN,2013,15/11/1995,032,auch,True,False
3,21301392,L3,E,resultat,2016/2017,2,NaN,NaN,NaN,NaN,2013,26/01/1994,054,nancy,True,False
4,21604149,L3,E,resultat,2016/2017,2,NaN,NaN,def,NaN,2016,05/10/1984,134,pamplona,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96764,21813431,L3,MAPI3,elmai6vm,2018/2019,1,13.000,NaN,adm,3.0,2018,08/05/1998,350,oujda,False,False
96765,21813168,L3,MAPI3,elmai6vm,2018/2019,1,14.200,NaN,adm,3.0,2018,07/06/1997,352,bejaia,False,False
96766,21407878,L3,MAPI3,elmai6vm,2018/2019,1,NaN,NaN,NaN,NaN,2014,26/05/1993,081,mazamet,True,False
96767,21606663,L3,MAPI3,elmai6vm,2018/2019,1,NaN,NaN,def,NaN,2016,25/11/1998,065,tarbes,True,False


### Et voila ! On a créé notre nouveau dataframe avec les infos de tous les PVs réunis
Maintenant, on l'exporte.

In [36]:
df.to_csv("fused_PVs_long_format.csv", index=False)